### The Transport Problem

```
Given a point (x,0) or (0,y) and an angle θ where the ray
enters a given lattice, find the entry points for each cell
the ray passes through.
```

Solution:<p>

The ray length between any two adjacent lines y=n and y=n+1,<br>
with $n {\in} {\mathbb Z}$ is directly given by the slope of the ray. To see this<br>
consider the equation for a line, $\ y = mx + b$, and $tan\ \theta$,<br>
where we understand `tan :: Angle -> Slope`.<p>

Clearly any line parallel to the ray has the same length as the ray<br>
from $y_n$ to $y_{n+1}$. This implies that adding any constant b does not<br>
have an effect on the ray length. Therefore given $y$, the value of $x$<br>
at the kth $y$, $x_k$ is given by:

<p>$x_k = \frac{k}{tan\ \theta}$<p>

ie. Letting $y$ vary along the non-negative integers gives the<br>
$x$-valued entry point at the kth cell. Taking the fractional part<br>
of the sum of the original x value with its transported value, $x_k$,<br>
yields the entry point relative to the particular cell.
<p>$x_c = fractional(x + x_k)$<p>

The $y$ values at the $x$-crossings are handled similarly:

<p>
$y_k = {k}*{tan\ \theta}$<br>
$y_c = fractional({y} + {k*tan\ \theta})$

In [18]:
fractional :: Double -> Double
fractional = snd.properFraction

xks θ = [k / tan θ | k<-[0..]] -- Should these be 1.. ?
yks θ = [k * tan θ | k<-[0..]]

kthX (x,y) θ k = fractional (x + (xks θ)!!k)

kthX (1/4, 0) (pi/8) 3 -- the xth-entry for the 3rd cell

0.49264068711928477

#### Test Ideas:
* Partials should sum to the Total
* Symmetry across the Total
* Boundaries, along the axes









### totalAttenuation

* array lengths should really be dependent on path lengths. These<br>
  will be better written as a conditional on limit: x<1000 && y<1000.
  

```
totalAttenuation (x,y) θ ary =
  let xcrossings = [ x + k / tan θ | k <- [0..999]] in
  let ycrossings = [ y + k * tan θ | k <- [0..999]] in
  walk xcrossings ycrossings θ ary

  where
    walk _ [] _ _ = 0
    walk [] _ _ _ = 0
    walk (x:xs) (y:ys) theta ary =
      let val = qArray (floor x, floor y) ary in
      case x < y of
      True -> -- bottom entry in frame
        val * rayLength (fractional x, 0) θ + walk xs (y:ys) θ ary
      False -> -- side entry in frame
        val * rayLength (0, fractional y) θ + walk (x:xs) ys θ ary
```

### totalRayLength

A reasonable application for the solution above is to write a function which calculates<br>
the total ray length through a given 2-dimensional lattice. It is assumed here that the<br>
lattice is oriented such that the ray always enters by crossing the face $y=0$. Because<br>
of this assumption, the `xcrossings` and `ycrossings` functions below must be initialized<br>
accordingly. An additional (temporary) assumption is that $\theta \leq \pi/2$. Altogether note that<br>
`totalRayLength` is dependent here on $x$ and $θ$ alone.

To be clear, the meaning of `xcrossings` is really the value of $x$ at each $y=k$, and the<br>
meaning of `ycrossings` is the value of $y$ at each $x=k$.

```
totalRayLength (x,y) θ size =
  let xcrossings = [ x + k / tan θ | k <- [0..size-1]] in
  let ycrossings = [ y + k * tan θ | k <- [0..size-1]] in

  walk xcrossings ycrossings θ

  where
    walk _ [] _ = 0
    walk [] _ _ = 0
    walk (x:xs) (y:ys) θ = case x < y of
      True ->
        rayLength (fractional x, 0) θ + walk xs (y:ys) θ
      False ->
        rayLength (0, fractional y) θ + walk (x:xs) ys θ
```